In [1]:
import sys
sys.path.append("../")

from SALib.sample import sobol
from network_model import *
from tqdm import tqdm
import numpy as np
import pickle
import lzma
import os

from astropy.stats import bayesian_blocks



import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({"font.size" : 15,
                     "figure.dpi" : 100, 
                     "grid.alpha" : 0.3, 
                     "axes.grid": True, 
                     "axes.axisbelow" : True,
                     "figure.figsize":(8,6),
                     "mathtext.fontset":"cm",
                     "xtick.labelsize": 14,
                     "ytick.labelsize": 14,
                     "axes.labelsize": 16, 
                     "legend.fontsize": 13.5})
USE_TEX = False
if USE_TEX:
    plt.rc("text", usetex=True)
    plt.rc("text.latex", preamble=r"""
     \usepackage{times}
     \usepackage{mathptmx}""")
else:
    plt.rc("text", usetex=False)
plt.rc("font", family="serif")

In [2]:
def gini(W):
    n = len(W)
    total = 0
    for i in range(n):
        total += sum(np.abs(W[i]-W))
    return total / (2 * n**2 * np.mean(W))

In [3]:
with open("../augmented_communities.pickle", "rb") as f:
    communities = pickle.load(f)

In [7]:
def category_1(data):
    decreasing = []
    for i in range(len(data)):
        d = data[i]
        mono_dec = True
        for row in d:
            if max(row[1:] - row[:-1]) > 0:
                mono_dec = False
                break
        if mono_dec:
            decreasing.append(i)
    return decreasing


def category_2(data, ignore):
    increase_less_initial = []
    for i in range(len(data)):
        if i not in ignore:
            d = data[i]
            if np.all(np.max(data[i], axis=1) == data[i][:,0]):
                increase_less_initial.append(i)
    return increase_less_initial


def category_3(data, ignore):
    surpass_end_poor = []
    for i in range(len(data)):
        if i not in ignore:
            if max(data[i].T[-1]) < 0.1:
                surpass_end_poor.append(i)
    return surpass_end_poor


def category_4(data, ignore):
    two_equilibria_less_initial = []
    for i in range(len(data)):
        if i not in ignore:
            if np.all(data[i].T[-1] < data[i].T[0]):
                two_equilibria_less_initial.append(i)
    return two_equilibria_less_initial


def category_5(data, ignore):
    two_equilibria_richer = []
    for i in range(len(data)):
        if i not in ignore:
            two_equilibria_richer.append(i)
    return two_equilibria_richer

In [8]:
directories = ["../data/sobol/concat_W_arrays", 
               "../data/sobol/concat_W_arrays_random", 
               "../data/sobol/concat_W_arrays_cpt"]

labels = ["CPT (Holme-Kim)", 
          "CPT (Random)", 
          "MPT (Holme-Kim)"]

# Analysis at agent level

In [ ]:
gini_cutoffs = None
percentage_results = {}

for dir_idx, D in enumerate(directories):
    
    print(f"Running analysis for {labels[dir_idx]}...")
    
    category_counts = {i:0 for i in range(7)}
    gini_coefficients = []
    
    for f in tqdm(os.listdir(D)):

        data = pickle.load(lzma.open(os.path.join(D,f)))
        
        cat1 = category_1(data)
        cat2 = category_2(data, cat1)
        cat3 = category_3(data, cat1+cat2)
        cat4 = category_4(data, cat1+cat2+cat3)
        cat5 = category_5(data, cat1+cat2+cat3+cat4)
        
        assert len(cat1+cat2+cat3+cat4+cat5) == len(data)

        gini_coefficients += [gini(data[i].T[-1]) for i in cat5]

        for i,cat in enumerate([cat1, cat2, cat3, cat4]):
            category_counts[i] += len(cat)

    if gini_cutoffs is None:
        sorted_coeffs = np.array(sorted(gini_coefficients))
        split_arrays = np.array_split(sorted_coeffs, 3)
        gini_cutoffs = [arr[-1] for arr in split_arrays]

    gini_categories = {cutoff:[] for cutoff in gini_cutoffs}
    for coeff in gini_coefficients:
        for key in gini_categories:
            if coeff <= key:
                gini_categories[key].append(coeff)
                break
 
    for i,cutoff in enumerate(gini_categories):
        category_counts[4+i] += len(gini_categories[cutoff])

    L = sum(category_counts.values())
    percentage_results[labels[dir_idx]] = [100*category_counts[i]/L for i in range(len(category_counts))]

    print("--- SUMMARY STATISTICS ---")
    print(f"CATEGORY 1: {percentage_results[labels[dir_idx]][0]:.3f}%")
    print(f"CATEGORY 2: {percentage_results[labels[dir_idx]][1]:.3f}%")
    print(f"CATEGORY 3: {percentage_results[labels[dir_idx]][2]:.3f}%")
    print(f"CATEGORY 4: {percentage_results[labels[dir_idx]][3]:.3f}%")
    print(f"CATEGORY 5 (gini <= {gini_cutoffs[0]:.4f}): {percentage_results[labels[dir_idx]][4]:.3f}%")
    print(f"CATEGORY 6 (gini <= {gini_cutoffs[1]:.4f}): {percentage_results[labels[dir_idx]][5]:.3f}%")
    print(f"CATEGORY 7 (gini <= {gini_cutoffs[2]:.4f}): {percentage_results[labels[dir_idx]][6]:.3f}%")
    print("\n\n")

Running analysis for CPT (Holme-Kim)...


100%|██████████| 10/10 [31:47<00:00, 190.71s/it]


--- SUMMARY STATISTICS ---
CATEGORY 1: 41.638%
CATEGORY 2: 6.852%
CATEGORY 3: 49.096%
CATEGORY 4: 0.473%
CATEGORY 5 (gini <= 0.9911): 0.647%
CATEGORY 6 (gini <= 0.9959): 0.647%
CATEGORY 7 (gini <= 0.9995): 0.647%



Running analysis for CPT (Random)...


100%|██████████| 10/10 [33:42<00:00, 202.22s/it]


--- SUMMARY STATISTICS ---
CATEGORY 1: 41.888%
CATEGORY 2: 6.956%
CATEGORY 3: 48.654%
CATEGORY 4: 0.480%
CATEGORY 5 (gini <= 0.9911): 0.807%
CATEGORY 6 (gini <= 0.9959): 0.650%
CATEGORY 7 (gini <= 0.9995): 0.565%



Running analysis for MPT (Holme-Kim)...


 10%|█         | 1/10 [12:27<1:52:05, 747.29s/it]

# Analysis at community level

In [6]:
with open("../augmented_communities.pickle", "rb") as f:
    communities = pickle.load(f)